In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import re
import os
from gensim import utils
from utils import check_super_phrase, dict_of_phrases
import math

# This is the pre-specified window size.
window = 4 * 4
REVIEW_FiELD = 9

# This is the directory which contains the curated phrases files.
#PHRASE_DIR = "curated_aspects_2"
PHRASE_DIR = "fine-grained-phrases"

# REVIEWS_FILE contains 1 sentence in every line
REVIEWS_FILE = "temp.txt"
OUTPUT_FILE = "temp_output.txt"

In [15]:
# The code searches for the brand names, aspect names and sentiment type in products

# reading the phrases from the phrase_dir in hierarchical way
phrase_dict, global_dict = dict_of_phrases(PHRASE_DIR)

# From a review extracting all the valid phrases and do a super phrase
# check on them
# Opening the reviews file and output file
infile = utils.smart_open(REVIEWS_FILE, 'rb')
outfile = utils.smart_open(OUTPUT_FILE, 'wb')

line_num = 0
for line in infile:
    line_num += 1
    sentence = utils.to_unicode(line).split("~")[REVIEW_FiELD]

    # By doing this words like don't and can't will become dont and cant
    sentence = sentence.replace("'", "")

    # Replacing dashes with spaces so as to prevent errors
    # in detecting word boundaries
    sentence = sentence.replace("-", " ")

    # Making the sentence lowercase
    sentence = sentence.lower().strip()

    toks = sentence.split(" ")
    toks1 = [word.strip() for word in toks]

    temp_dict = dict()
    
    if len(toks1) > 1:
        # what is the importance of this condition ?
        if line_num % 1000 is 0:
            print 'Processed %i lines' % line_num

        # Modifying the code to incorporate unigrams
        for i, start_token in enumerate(toks1[0:]):
            
            # starting the range from zero to incorporate unigrams
            for j in range(0, window):
                if i + j + 1 <= len(toks1):
                    new_string = " ".join(toks1[i:i + j + 1])
                    if new_string in global_dict:
                        temp_dict[new_string] = global_dict[new_string]

    # Keeping only superphrases                    
    line_phrase_dict = check_super_phrase(temp_dict.keys(), temp_dict, window)

    # sorting the phrases in descending order of coherence
    temp_list = sorted(line_phrase_dict.items(), key=lambda x: x[1], reverse=True)
    string = " ".join(toks1)

    line_phrase_list = list()
    # filtering out the non-overlapping phrases in line and adding unigrams in line_phrase_list
    for phrase, _ in temp_list:
        if len(phrase.split(" ")) is 1:
            line_phrase_list.append(phrase)
        else:
            string = re.sub(r'\b' + phrase + r'\b', "_".join(phrase.split()), string)

    # Tokenize the string by whitespace. SEARCH the tokens for underscores.
    # if "_" is found, REPLACE the "_" by " ", to get the desired phrase.
    toks = string.split(" ")
    for word in toks:
        if "_" in word:
            word = word.replace("_", " ")
            line_phrase_list.append(word)

    print >> outfile, utils.to_utf8(line.strip()), '~',

    # Detecting the negation in a sentence
    transformed = re.sub(
        r'\b(?:not|cannot|could\ be|can\ be|yet\ to|never|no|nothing|nowhere|noone|none|havent|hasnt|hadnt|cant|couldnt|without|shouldnt|wont|wouldnt|dont|doesnt|dosnt|didnt|isnt|arent|aintnever|werent)\b[\w\s_]+',
        lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0), count=7), string)

    # Detecting second kind of negation in sentence
    transformed2 = re.sub(
        r'[\w\s_]+\b(?:except)\b',
        lambda match: re.sub(r'(\s*)(\w+)', r'\1NEG_\2', match.group(0), count=5), transformed)
    
    print transformed2
    
    transformed_toks = transformed2.split(" ")

    # Search for the corresponding phrase in the aspect and sentiment category
    valid_tuples = list()
    
    # line_phrase_list contains the tags identified for the review
    for tag in line_phrase_list:
        for category in phrase_dict:
            tag_underscore = tag.replace(" ", "_")
            tag_new = tag_underscore

            # If we detect negations in transformed tokens, 
            # then include "NEG_" string at the start of the corresponding phrase 
            if "sentiments" == category or "aspects-sentiments" == category:
                if 'NEG_' + tag_underscore in transformed_toks:
                    tag_new = 'NEG_' + tag_underscore
            
            for cat_type in phrase_dict[category]:
                if tag in phrase_dict[category][cat_type]:

                    # Dealing with the case of "aspects-sentiments" directory
                    if category == "aspects-sentiments":
                        aspect_type = cat_type.split("_")[0]
                        sent_type = cat_type.split("_")[1]

                        # Changing the polarity of the sentiment due to negation detection
                        if tag_new == 'NEG_' + tag_underscore:
                            if sent_type == "positive":
                                sent_type = "negative"
                            elif sent_type == "negative":
                                sent_type = "positive"
                            elif sent_type == "most-positive":
                                sent_type = "most-negative"
                            elif sent_type == "most-negative":
                                sent_type = "most-positive"
                            elif sent_type == "neutral":
                                sent_type = "negative"

                        aspect_tuple = (category.split("-")[0], aspect_type, tag_new)
                        sent_tuple = (category.split("-")[1], sent_type, tag_new)
                        print >> outfile, (aspect_tuple, sent_tuple), "~",
        
                    # For "aspects" and "sentiments" class of keywords
                    else:
                        tag = tag.replace(" ", "_")

                        try:
                            tag_index = toks.index(tag)
                        except ValueError:
                            print "line is"

                        if tag_new == 'NEG_' + tag_underscore:
                            if cat_type == "positive":
                                cat_type = "negative"
                            elif cat_type == "negative":
                                cat_type = "positive"
                            elif cat_type == "most-positive":
                                cat_type = "most-negative"
                            elif cat_type == "most-negative":
                                cat_type = "most-positive"
                            elif cat_type == "neutral":
                                cat_type = "negative"

                        valid_tuples.append((category, cat_type, str(tag_new), tag_index))

    # find the closest sentiment to an aspect
    sorted_aspect_sent_list = sorted(valid_tuples, key=lambda x: x[3])
    
    for k, aspect_tuple in enumerate(sorted_aspect_sent_list):
        if aspect_tuple[0] == 'aspects':
            sent_tuple = min(sorted_aspect_sent_list,
                             key=lambda x: abs(x[3] - aspect_tuple[3]) if (x[0] == 'sentiments') else 1000)
            if sent_tuple[0] == "sentiments":
                if math.fabs(aspect_tuple[-1] - sent_tuple[-1]) < 15:
                    print >> outfile, (aspect_tuple[0:-1], sent_tuple[0:-1]), "~",

    print >> outfile, '\n',

infile.close()
outfile.close()

performance is not NEG_up_to_the_mark
back_camera very very not_good
with this new phone i have yet NEG_to NEG_have NEG_any NEG_errors NEG_or NEG_crashes
i have had no_lag and heating_problem with this device till now
the_display_is not NEG_better
imaging youd be unhappy_with
apps and games_are smooth without NEG_a_lag
cannot NEG_open NEG_it NEG_to NEG_replace_the_battery
battery isnt NEG_that NEG_great
battery isnt NEG_awesome
volume can NEG_be NEG_better
NEG_everything NEG_is_good NEG_except processor
i have yet NEG_to NEG_have NEG_any NEG_issues NEG_or NEG_crashes


In [16]:
import pprint
#pprint.pprint(phrase_dict)